<a href="https://colab.research.google.com/github/Kinds-of-Intelligence-CFI/comparative-object-permanence/blob/measurement-layout/analysis/measurement-layouts/object_permanence_measurement_layout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparative Object Permanence Measurement Layouts

Authors: K. Voudouris, J. Burden, J. Hernández-Orallo

## INIT

In [ ]:
!pip install pymc
!pip install numpy
!pip install arviz
!pip install erroranalysis

In [ ]:
import pymc as pm
import numpy as np
import arviz as az
import random as rm
from scipy import stats
import pandas as pd
import erroranalysis as ea
import matplotlib.pyplot as plt
import gc
from google.colab import files
import math
import seaborn as sns
print(f"Running on PyMC v{pm.__version__}")